In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import os
import simplekml

from datetime import datetime, timedelta
from matplotlib.backends.backend_pdf import PdfPages

In [44]:
#Icing Testing
file = "2023-01-26 Icing//23-01-25-09-44-01_FLY011.csv"
file = "2023-01-26 Icing//23-01-25-10-24-45_FLY013.csv"
file = "2023-01-26 Icing//23-01-25-10-36-51_FLY014.csv"
file = "2023-01-26 Icing//23-01-25-10-45-43_FLY015.csv"
file = "2023-01-26 Icing//23-01-25-10-58-55_FLY016.csv"
file = "2023-01-26 Icing//23-01-25-11-08-29_FLY017.csv"
file = "2023-01-26 Icing//23-01-25-01-45-22_FLY018.csv"

flight_name = os.path.splitext(os.path.basename(file))[0]
pdf_name = flight_name + ".pdf"
kmz_name = flight_name + ".kmz"
print(pdf_name)


23-01-25-01-45-22_FLY018.pdf


In [45]:
#import the csv file
df = pd.read_csv(file)
df.fillna(method = 'bfill', inplace = True)
df.fillna(value = 0, inplace = True)
#print(df.head())
#print(df.tail())
#fill na with zeros


C:\Users\hastiet\AppData\Local\Temp\ipykernel_26944\2088013907.py:2: DtypeWarning: Columns (49,50,123,124,125,126,127,128,129,130,131,132,218) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


In [46]:
#for debugging. Look at the columns we imported.
col_names = list(df.columns)
#drop the last four columns becuase they're not data
col_names = col_names[0:-4]

#for col in col_names: 
#    print(col)

In [47]:
#add a datetime column to the df for 
#this cell is for logs that didn't have a GPS signal. It used an assumed clock tick rate of 
#3843605.539
#drop the first row since it seems to have an invalid timestamp
df.drop(df.head(1).index,inplace=True) 
#df['times'] = pd.to_datetime(df['GPS:dateTimeStamp'], format="%Y-%m-%dT%H:%M:%SZ")

df['times'] = (df['Clock:Tick#'] - df['Clock:Tick#'].min())/3843605.539
#print(df.tail())

#print(df['times'])

In [48]:
#loop through each column and plot the data against time. 
#save as a PDF with the same name as the CSV file.
with PdfPages(pdf_name) as pdf:
     #add a graph of lat/long
    fig = plt.plot(df['GPS:Long'],df['GPS:Lat'])
    plt.title("Lat vs Long")
    plt.grid()
    #plt.show()
    pdf.savefig()
    plt.close()
    
    for col in col_names: 
        #print(col)
        fig = plt.plot(df['times'],df[col])
        plt.title(col)
        plt.grid()
        #plt.show()
        pdf.savefig()
        plt.close()
        
        
print("....Done!")
        
    

....Done!


In [156]:
#make a KMZ of the flight path
#Obviously this only works for logs that have GPS data.
#build the description string for the kml file:
analysis_date = datetime.now()
description_string = "KML File created: %s"%analysis_date.strftime('%b %d, %Y')
#description_string += "\n\nSource Data File: %s"%filename[0].split('\\')[-1]
print(description_string)
line_color = 'ff00ff00'  # Green


kml = simplekml.Kml()
kml.document.description = description_string

    

coords = []
for coord in zip(df['GPS:Long'],df['GPS:Lat'], df['GPS:heightMSL']):
    #print(coord)
    coords.append(coord)
print(len(coords))



#folder for the tracks
#fol = approach_folder.newfolder(name=str(flights[flight]['model'][0]))
path = kml.newlinestring(name=str(flight_name))
path.coords = coords    
path.altitudemode = simplekml.AltitudeMode.absolute
path.style.linestyle.color = line_color

    
#print(kml.kml())
kml.savekmz(kmz_name)


KML File created: Jun 05, 2021
23580


In [ ]:
#backup/Testing Cells

In [ ]:
#add a datetime column to the df for 
#this cell will only work for logs that had a GPS signal.
#drop the first row since it seems to have an invalid timestamp
df.drop(df.head(1).index,inplace=True) 
df['times'] = pd.to_datetime(df['GPS:dateTimeStamp'], format="%Y-%m-%dT%H:%M:%SZ")
df['times'] = df['times'] - timedelta(hours=4)

#print(df['times'])

In [ ]:
#If we don't have GPS, we need to estimate the clock speed. 
#the intent here is to load a log WITH GPS, and use the GPS time stamp to figure out the clock speed. 

#figure out how many ticks per second
print("Log Duration:")
duration = df['times'].max() - df['times'].min()
print(duration) 

print("Number of Ticks:")
ticks = df['Clock:Tick#'].max() - df['Clock:Tick#'].min()
print(ticks)

print("Ticks per second")
seconds = duration.total_seconds()
freq = ticks/seconds
print(freq)


#based on the logs I took with the M300 outside, the clock rate of the log averages out to: 
#3843605.539 ticks per second.
# so like.. 3.8MHz. 
#this 3.8MHz number is used above to give a time estimation for M300 logs that didn't have enough GPS data 
#(i.e. in the wind tunnel)
